# JijModeling 1.8.0 Release Note

## `jijmodeling.Interpreter`

- `jijmodeling` supports stand-alone compile
  - [`jijmodeling_transpiler`](https://pypi.org/project/jijmodeling-transpiler/) or JijZept are not required now!
- This evaluates an expression in `jijmodeling`, e.g. `x + y` into `ommx.v1.Function`
- Also evaluates entire `jijmodeling.Problem` into `ommx.v1.Instance`

### Substitute instance data

JijModeling AST

In [2]:
import jijmodeling as jm
import ommx.v1

In [3]:
# Define coefficients
a = jm.Placeholder("a", ndim=1)
n = a.len_at(0, latex="n")
i = jm.Element("i", belong_to=n)

# Decision variables
x = jm.BinaryVar("x", shape=(n,))

# Linear function
f = jm.sum(i, a[i] * x[i])
f

sum(Element(name='i', belong_to=(NumberLit(value=0), ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n'))), Placeholder(name='a', ndim=1)[Element(name='i', belong_to=(NumberLit(value=0), ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n')))] * BinaryVar(name='x', shape=[ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n')])[Element(name='i', belong_to=(NumberLit(value=0), ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n')))])

can be compiled by `Interpreter.eval_expr`

In [7]:
interpreter = jm.Interpreter({"a": [1, 2, 3]})
f_ = interpreter.eval_expr(f)
f_

Function(x0 + 2*x1 + 3*x2)

### Eval Problem into ommx.v1.Instance

### Limitations

- `jijmodeling.CustomPenaltyTerm` is not supported yet.
  - This will be supported in next version with OMMX update.